# Configuring Ubuntu 14.04.5 or 16.04 to run LXC

## Installing LXC

These instructions have been tested on Ubuntu 16.04 LTS and Ubuntu 14.04.5 LTS. They are likely not to work with other distributions of Linux. If you do not have access to a Ubuntu Linux system, you may also follow along using a Ubuntu virtual machine running in VirtualBox or an equivalent virtual machine framework. Each LXC container will be running Ubuntu 14.04.5 LTS.

### Ubuntu 14.04.5 Only: Update LXC PPA
The version of LXC available in the Ubuntu repositories is fairly old. Get an newer version by using the LXC team's PPA.

Run

```console
user@host:~$ sudo add-apt-repository ppa:ubuntu-lxc/stable
```

and press "Enter" to continue when prompted.

### Ubuntu 14.04.5 and Ubuntu 16.04: Update Repositories and Install

Next refresh your repository information and install LXC and Git by running

```console
user@host:~$ sudo apt-get update
user@host:~$ sudo apt-get install lxc lxd git
```

### Add LXC Networking Support

Update /etc/lxc/lxc-usernet by running: 

```console
user@host:~$ sudo nano /etc/lxc/lxc-usernet
```

Update the file to match the following, replacing user-name with your current user name.

```console
# USERNAME TYPE BRIDGE COUNT
user-name veth lxcbr0 10
```

When finished editing, use Ctrl+o to Output (save) the file, and Ctrl+x to Exit from nano. 

### Update Subuid and Subguid
The default starting subgid and subuid has changed between the packages in Ubuntu 14.04 LTS and Ubuntu 16.04 LTS. To ensure a common baseline, update the subgid and subuid config files as shown below:

```console
user@host:~$ sudo nano /etc/subuid
```

Modify the line associated with your username to the following, replacing "username" with your actual username.

```
username:165536:65536
```

Modify /etc/subgid as follows:

```console
user@host:~$ sudo nano /etc/subgid
```

Modify the line associated with your username to the following, replacing "username" with your actual username.

```
username:165536:65536
```

The updates to these two files will ensure LXC can correctly map container user IDs and group IDs to ranges usable by the scoring infrastructure.


Do note that these updates will break any containers created using a subgid and subuid base of 100000.


### Set up default LXC configuration
Create a configuration directory for LXC and copy a default configuration file into it by running

```console
user@host:~$ mkdir -p ~/.config/lxc
user@host:~$ cp /etc/lxc/default.conf ~/.config/lxc/default.conf
```

Next you'll need to set up user and group ID mappings. Edit the default LXC configuration file using:

```console
user@host:~$ nano ~/.config/lxc/default.conf
```

To be able to run containers generated in either Ubuntu 14 or Ubuntu 16, update the default LXC configuration file using the following mappings:

```console
lxc.id_map = u 0 165536 65536
lxc.id_map = g 0 165536 65536
```

### Update .local directory to be able to use defaul LXC prefix
LXC needs other users and groups, specifically the other users and groups executing inside containers, to be able to execute their own files. Make this possible by adding the execute flag to your .local directory:

```console
user@host:~$ mkdir ~/.local/
user@host:~$ chmod -R o+x ~/.local/
```

### Update user groups to be able to use LXD
Users must be in the lxd group to use the lxc import command, otherwise they will receive the following error:

```console
Permission denied, are you in the lxd group?
```

Use the following command to add a user to the lxd group, replacing username with the actual user name:

```console
user@host:~$ sudo usermod -aG lxd username


### Logout so changes can take effect
You need to completely log out of the system, not just the current terminal. 

### Log back in to test
Test that you can create, run, and attach to unprivileged LXC containers by logging back in and running:

```console
user@host:~$ lxc-create -t download -n my-container -- -d ubuntu -r trusty -a amd64
user@host:~$ lxc-start -n my-container
user@host:~$ lxc-attach -n my-container
```

If everything is configured correctly, you should be logged in as the root user of the container, at a prompt that looks like this:

```console
root@my-container:/# 
```

Type exit and hit enter to get back out of the container. Shut it down and delete the container file by running:

```console 
user@host:~$ lxc-stop -n my-container
user@host:~$ lxc-destroy -n my-container
```

You are now ready to run the hurdle build scripts.

## Initializing LXD
LXD will be used to robustly import and export participant images. 

To use it, it needs to be initialized once after installation. Initialize it by running: 

```console 
sudo lxd init
```

This command will prompt you for a series of responses. Use the following:

```console
Name of the storage backend to use (dir or zfs) [default=dir]: dir

Would you like LXD to be available over the network (yes/no) [default=no]? no

Would you like stale cached images to be updated automatically (yes/no) [default=yes]? yes

Would you like to create a new network bridge (yes/no) [default=yes]? yes
```

**At this point, Ubuntu 14 and Ubuntu 16 behave differently.**

**Ubuntu 14** will continue on with the following:

```console
What should the new bridge be called [default=lxdbr0]? lxdbr0

What IPv4 subnet should be used (CIDR notation, “auto” or “none”) [default=auto]? 

What IPv6 subnet should be used (CIDR notation, “auto” or “none”) [default=auto]? none

LXD has been successfully configured.
```

**Ubuntu 16** will open up a text menu, asking you the following:

```console
                Configuring lxd
Would you like to setup a network bridge for LXD containers now?  
<Yes>

Bridge interface name:
lxdbr0
<Ok>

Do you want to setup an IPv4 subnet?
<Yes>

IPv4 address: 
accept the randomly generated, valid default. Example would be
10.97.166.1
<Ok>

IPv4 CIDR mask:
24
<Ok>

First DHCP address:
accept the default based on your IPv4 address choice from earlier. Example would be
10.97.166.2
<Ok>

Last DHCP address:
accept the default based on your IPv4 address choice from earlier. Example would be
10.97.166.254
<Ok>

Max number of DHCP clients: 
252
<Ok>

Do you want to NAT the IPv4 traffic?
<Yes>

Do you want to setup an IPv6 subnet? 
<No>

LXD has been successfully configured.
```